# 💻 Installing necessary libraries
- `boto3` AWS operations
- `efficientnet` Neural network used for image classification:

In [ ]:
!pip install boto3
!pip install efficientnet

# 🔒 Retrieving AWS Credentials and configuring S3 connection

In [7]:
from google.colab import drive
import boto3
drive.mount('/content/drive')


def get_credential_value(lines: list, line_number: int):
    return lines[line_number].split('=')[1].strip()

CREDENTIALS_PATH = '/content/drive/MyDrive/vscodethemes/credentials.txt'

with open(CREDENTIALS_PATH, 'r') as credentials_file:
    lines = credentials_file.readlines()

    aws_access_key_id = get_credential_value(lines, 0)
    aws_secret_access_key = get_credential_value(lines, 1)
    aws_region_name = get_credential_value(lines, 2)
    bucket_name = get_credential_value(lines, 3)

s3_client = boto3.client(
    service_name='s3',
    region_name=aws_region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🞜 Downloading the themes stored on S3 

In [ ]:
import os
from tqdm import tqdm

DATA_PATH = "/content/data"

try:
   os.makedirs(DATA_PATH)
except FileExistsError:
   pass

def count_s3_bucket_objects(s3):
    paginator = s3_client.get_paginator('list_objects')
    counter = 0

    for page in paginator.paginate(Bucket=bucket_name):
        counter += len(page['Contents'])

    return counter


total_objects = count_s3_bucket_objects(s3_client)

def get_all_s3_objects(s3, **base_kwargs):
    continuation_token = None
    while True:
        list_kwargs = dict(MaxKeys=1000, **base_kwargs)
        if continuation_token is not None:
            list_kwargs["ContinuationToken"] = continuation_token

        themes_response = s3.list_objects_v2(**list_kwargs)
        
        yield from themes_response.get("Contents", [])
        
        if not themes_response.get("IsTruncated"):
            break
        
        continuation_token = themes_response.get("NextContinuationToken")

for theme_object in tqdm(get_all_s3_objects(s3_client, Bucket=bucket_name), total=total_objects, desc='Downloading files'):
    theme_official_name, file_name = theme_object["Key"].split('/')
    folder_local_path = f"{DATA_PATH}/{theme_official_name}"
    os.makedirs(folder_local_path, exist_ok=True)
    s3_client.download_file(bucket_name, theme_object["Key"], f"{folder_local_path}/{file_name}")
